In [1]:
import pandas as pd
import pypyodbc as odbc

In [2]:
# product = input()
product = 'iphone 15 pro max'

In [7]:
drive_name = 'SQL SERVER'
server_name = 'DESKTOP-29355SM\SQLEXPRESS'
database_name = 'Bank'

connection_string = f"""
    DRIVER={{{drive_name}}};
    SERVER={server_name};
    DATABASE={database_name};
    Trust_Connection=yes;
"""

conn = odbc.connect(connection_string)

In [9]:
cursor = conn.cursor()

In [13]:
cursor.execute("""
Create table Persons (
    PersonID int,
    LastName varchar(255),
    FirstName varchar(255),
    Address varchar(255),
    City varchar(255)
)
""")

In [14]:
conn.commit()

In [ ]:
# df = pd.read_sql('Select * from account', conn)

# Tiki

#### Product info

In [15]:
tiki_product_info = pd.read_csv(r'..\Data\Tiki\%s_info.csv'%('_'.join(product.split(' '))))

In [16]:
tiki_product_info = tiki_product_info[['Name', 'Brand', 'Quantity', 'Reviews count', 'Star rating', 
                                       'Price', 'Crawl date', 'Shop name', 'Shop rating']]

In [ ]:
tiki_product_info['Platform ']

In [17]:
tiki_product_info

,Name,Brand,Quantity,Reviews count,Star rating,Price,Crawl date,Shop name,Shop rating
0,Apple iPhone 15 Pro Max,Apple,691,96,5.0,31.990.000,2024-01-11,Tiki Trading,4.7
1,Apple iPhone 15 Pro Max,Apple,726,100,5.0,31.990.000,2024-01-18,Tiki Trading,4.7
2,Apple iPhone 15 Pro Max,Apple,740,103,5.0,31.990.000,2024-01-27,Tiki Trading,4.7


#### Reviews

In [20]:
tiki_reviews = pd.read_csv(r'..\Data\Tiki\%s_reviews.csv'%('_'.join(product.split(' '))))

In [21]:
tiki_reviews

,Reviewer name,Content,Rating,Rating date,Crawl date,Shop name
0,muasach,"22/9 đúng 00h lên topzone, TGDĐ, FPT Shop, đặt...",Cực kì hài lòng,Đánh giá vào 3 tháng trước,2024-01-11,Tiki Trading
1,Vũ thị hường,Đam mê táo k thể bỏ qua Iphone 15promax vì: \r...,Cực kì hài lòng,Đánh giá vào 3 tháng trước,2024-01-11,Tiki Trading
2,BẢO NGÔ,[Cuộc gọi lúc giữa đêm]…\r\n\r\nKhông phải bon...,Cực kì hài lòng,Đánh giá vào 3 tháng trước,2024-01-11,Tiki Trading
3,NGÔ QUỐC KHÁNH,Tiki giao lúc 1h sáng. Áp mã 500K và giảm 2tr ...,Cực kì hài lòng,Đánh giá vào 3 tháng trước,2024-01-11,Tiki Trading
4,Hung Trieu,Rất tin tưởng Tiki khi đặt mua điện thoại tại ...,Cực kì hài lòng,Đánh giá vào 2 tháng trước,2024-01-11,Tiki Trading
...,...,...,...,...,...,...
191,vu thang,Thấy bảo nhận hàng phải có mã otp mới nhận đc ...,Rất không hài lòng,Đánh giá vào 2 tháng trước,2024-01-18,Tiki Trading
192,Việt Dũng Đoàn,Hàng ko cho đổi trả khi đổi ý,Rất không hài lòng,Đánh giá vào 3 tháng trước,2024-01-18,Tiki Trading
193,Thu Nga,Chưa thấy hoàn astra cho mình,Không hài lòng,Đánh giá vào 3 tháng trước,2024-01-18,Tiki Trading
194,Nguyễn Quang,NaN,Hài lòng,Đánh giá vào 2 tháng trước,2024-01-18,Tiki Trading


- Loại bỏ 'Reviewer name'
- 'Rating' chuyển thành thang đo từ 1 đến 5 hay 3 mức độ (tích cực, trung tính và tiêu cực)
- Biến đổi cột 'Rating date' thành ngày cụ thể thông qua 'Crawl date'
- Xử lí duplicate

In [24]:
from underthesea import sentiment

In [30]:
tiki_reviews[['Content', 'Rating']]

,Content,Rating
0,"22/9 đúng 00h lên topzone, TGDĐ, FPT Shop, đặt...",Cực kì hài lòng
1,Đam mê táo k thể bỏ qua Iphone 15promax vì: \r...,Cực kì hài lòng
2,[Cuộc gọi lúc giữa đêm]…\r\n\r\nKhông phải bon...,Cực kì hài lòng
3,Tiki giao lúc 1h sáng. Áp mã 500K và giảm 2tr ...,Cực kì hài lòng
4,Rất tin tưởng Tiki khi đặt mua điện thoại tại ...,Cực kì hài lòng
...,...,...
191,Thấy bảo nhận hàng phải có mã otp mới nhận đc ...,Rất không hài lòng
192,Hàng ko cho đổi trả khi đổi ý,Rất không hài lòng
193,Chưa thấy hoàn astra cho mình,Không hài lòng
194,NaN,Hài lòng


In [27]:
tiki_reviews['Content'][0]

'22/9 đúng 00h lên topzone, TGDĐ, FPT Shop, đặt mà không được, load trang liên tục 30 phút toàn báo lỗi, đến khi load được thì báo hết suất đặt cọc. Đợi tới 29/9 cũng đúng 00h lên tiki đặt, cũng vô đặt hoài tầm 30 phút mà không được, lúc đặt được thì nó báo địa chỉ không hỗ trợ, mặc dù địa chỉ này mình đặt mỗi ngày có bị gì đâu, khúc này là quạo rồi nha, bực quá tắt máy đi ngủ tiếp, mà tức quá thì làm sao ngủ được, mở điện thoại lên đặt tiếp, ơn trời lần này đặt được, mừng quá cũng hết ngủ luôn, nói chung là một đêm mất ngủ, cuối cùng được tiki giao trưa 29/9 luôn, quá ổn áp. Thích màu titan tự nhiên hơn, mà không đặt được, đặt được màu xanh là mừng húm rồi, thôi dù sao cũng đeo ốp lưng, nên màu nào cũng được, à quên mình thanh toán thẻ + voucher tiki nên được giảm 2tr5, còn 32tr490 cho bản 256GB (15 promax) quá êm. Thu gọn'

In [26]:
sentiment(tiki_reviews['Content'][0])

'positive'

# Lazada

# Websites

#### Tinhte

In [8]:
tinhte_content = pd.read_csv(r'..\Data\TinhTe\TinhTe_Iphone15pm_InfoPosts2.csv')

# Youtube